In [6]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

In [34]:

# Definir el rango de valores para el eje X
t = np.linspace(0, 13, 1000)

# Crear una tendencia lineal desde 0 hasta 1
trend = np.linspace(0, 1, 1000)

# Crear un componente cíclico con una función sinusoidal
cyclical = 0.5 * np.sin(2 * np.pi * t / 6)

# Crear un componente estacional con una función sinusoidal más suave
seasonal = 0.1 * np.sin(2 * np.pi * t * 1.95)

# Crear un componente de fluctuaciones irregulares con ruido aleatorio
irregular = 0.05 * np.random.randn(1000)

# Crear la figura
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=trend, mode='lines', name='Trend', line=dict(color='black')))
fig.add_trace(go.Scatter(x=t, y=trend + cyclical, mode='lines', name='Cyclical', line=dict(color='green')))
fig.add_trace(go.Scatter(x=t, y=trend + cyclical + seasonal, mode='lines', name='Seasonal', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=t, y=trend + cyclical + seasonal + irregular, mode='lines', name='Irregular Fluctuations', line=dict(color='purple')))


# Configurar el diseño del gráfico
fig.update_layout(
    title='Irregular Fluctuations Component of Time Series',
    xaxis_title='Year',
    yaxis_title='Value',
    xaxis=dict(range=[0, 13]),
    yaxis=dict(range=[-0.2, 1.2])
)
fig.update_traces(selector = dict(name = 'Irregular Fluctuations'), line = dict(width=1))
# Mostrar la gráfica
fig.show()

EXTRACCION DE DATOS


In [35]:
# Construir el DataFrame
df = pd.DataFrame({
    'Year': t,
    'Trend': trend,
    'Cyclical': cyclical,
    'Seasonal': seasonal,
    'Irregular Fluctuations': irregular,
    'Total': trend + cyclical + seasonal + irregular
})

# Mostrar las primeras filas de la tabla
display(df)

# Guardar en un archivo CSV
df.to_csv('time_series_data.csv', index=False)

,Year,Trend,Cyclical,Seasonal,Irregular Fluctuations,Total
0,0.000000,0.000000,0.000000,0.000000,0.060107,0.060107
1,0.013013,0.001001,0.006813,0.015876,0.016344,0.040035
2,0.026026,0.002002,0.013626,0.031350,0.040933,0.087910
3,0.039039,0.003003,0.020435,0.046028,-0.014068,0.055398
4,0.052052,0.004004,0.027241,0.059539,-0.018257,0.072527
...,...,...,...,...,...,...
995,12.947948,0.995996,0.418749,0.099996,0.049046,1.563787
996,12.960961,0.996997,0.422433,0.098877,0.050336,1.568644
997,12.973974,0.997998,0.426039,0.095250,0.003512,1.522800
998,12.986987,0.998999,0.429566,0.089207,0.052773,1.570545


In [36]:
# Cargar los datos desde el DataFrame generado previamente
df = pd.read_csv('time_series_data.csv')

TRANSFORMADA DE FOURIER

In [37]:


# Extraer la señal total y el tiempo
signal = df['Total'].values
t = df['Year'].values
dt = t[1] - t[0]  # Paso de tiempo

# Aplicar la Transformada de Fourier
fft_values = np.fft.fft(signal)
frequencies = np.fft.fftfreq(len(t), d=dt)

# Filtrar las frecuencias en el rango deseado (-4 a 4)
mask = (frequencies >= -4) & (frequencies <= 4)
filtered_frequencies = frequencies[mask]
filtered_fft_magnitude = np.abs(fft_values[mask])

# Crear la figura con Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=filtered_frequencies, 
    y=filtered_fft_magnitude, 
    mode='lines', 
    name='FFT Magnitude',
    line=dict(color='blue')
))

# Configurar el diseño del gráfico con fondo blanco y rango de X ajustado
fig.update_layout(
    title='Espectro de Fourier de la Serie Temporal',
    xaxis_title='Frecuencia (Hz)',
    yaxis_title='Magnitud',
    template='plotly_white',  # Fondo blanco
    xaxis=dict(range=[-4, 4], showgrid=True),
    yaxis=dict(showgrid=True)
)

# Mostrar la gráfica
fig.show()

CON SCATTER

In [38]:
# Crear la figura con Scatter plot en lugar de líneas
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=filtered_frequencies, 
    y=filtered_fft_magnitude, 
    mode='markers',  # Scatter plot con puntos
    name='FFT Magnitude',
    marker=dict(color='blue', size=5)
))

# Configurar el diseño del gráfico con fondo blanco y rango de X ajustado
fig.update_layout(
    title='Espectro de Fourier de la Serie Temporal',
    xaxis_title='Frecuencia (Hz)',
    yaxis_title='Magnitud',
    template='plotly_white',  # Fondo blanco
    xaxis=dict(range=[-4, 4], showgrid=True),
    yaxis=dict(showgrid=True)
)

# Mostrar la gráfica
fig.show()

AUMENTANDO RESOLUCION

In [39]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Cargar los datos desde el DataFrame generado previamente
df = pd.read_csv('time_series_data.csv')

# Extraer la señal total y el tiempo
signal = df['Total'].values
t = df['Year'].values
dt = t[1] - t[0]  # Paso de tiempo

# Aumentar la resolución de la FFT con Zero-Padding (interpola más puntos)
N = len(signal) * 4  # Multiplica por 4 la resolución (ajustable)
fft_values = np.fft.fft(signal, n=N)
frequencies = np.fft.fftfreq(N, d=dt)

# Filtrar las frecuencias en el rango deseado (-4 a 4)
mask = (frequencies >= -4) & (frequencies <= 4)
filtered_frequencies = frequencies[mask]
filtered_fft_magnitude = np.abs(fft_values[mask])

# Crear la figura con Scatter plot (más puntos)
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=filtered_frequencies, 
    y=filtered_fft_magnitude, 
    mode='markers',  # Scatter plot con puntos
    name='FFT Magnitude',
    marker=dict(color='blue', size=4, opacity=0.7)  # Puntos más pequeños y semitransparentes
))

# Configurar el diseño del gráfico con fondo blanco y rango de X ajustado
fig.update_layout(
    title='Espectro de Fourier con Mayor Resolución',
    xaxis_title='Frecuencia (Hz)',
    yaxis_title='Magnitud',
    template='plotly_white',  # Fondo blanco
    xaxis=dict(range=[-4, 4], showgrid=True),
    yaxis=dict(showgrid=True)
)

# Mostrar la gráfica
fig.show()




In [40]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Cargar los datos desde el DataFrame generado previamente
df = pd.read_csv('time_series_data.csv')

# Extraer la señal total y el tiempo
signal = df['Total'].values
t = df['Year'].values
dt = t[1] - t[0]  # Paso de tiempo

# Aumentar la resolución de la FFT con Zero-Padding (interpola más puntos)
N = len(signal) * 4  # Multiplica por 4 la resolución (ajustable)
fft_values = np.fft.fft(signal, n=N)
frequencies = np.fft.fftfreq(N, d=dt)

# Filtrar las frecuencias en el rango deseado (-4 a 4)
mask = (frequencies >= -4) & (frequencies <= 4)
filtered_frequencies = frequencies[mask]
filtered_fft_magnitude = np.abs(fft_values[mask])

# Crear la figura con una línea continua
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=filtered_frequencies, 
    y=filtered_fft_magnitude, 
    mode='lines',  # Línea continua
    name='FFT Magnitude',
    line=dict(color='blue', width=2)
))

# Configurar el diseño del gráfico con fondo blanco y rango de X ajustado
fig.update_layout(
    title='Espectro de Fourier con Línea Continua',
    xaxis_title='Frecuencia (Hz)',
    yaxis_title='Magnitud',
    template='plotly_white',  # Fondo blanco
    xaxis=dict(range=[-4, 4], showgrid=True),
    yaxis=dict(showgrid=True)
)

# Mostrar la gráfica
fig.show()
